# Embedtree2
Notebook to compile the so far research of nvgtt.

In [165]:
import requests
import nltk
from bs4 import BeautifulSoup
import re
import urllib
import networkx as nx
import sys

from ThreadPool import ThreadPool

from wikipydia import dict_storage, wikidb
DictStorage = dict_storage.DictStorage

In [166]:
wiki_db = wikidb.WikiDb()

## Main()

In [167]:
last_percent_reported = None
def download_progress_hook(count, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

In [168]:
wikisyn = DictStorage("wikisyn") #Storage for link synoms
#href_to_pageid = DictStorage("href_to_pageid") #Storage lookup table of hrefs and pageids
#pageid_to_title = DictStorage("pageid_to_title") #Storage for page titles
#pageid_to_href = DictStorage("pageid_to_href") #Storage lookup table of pageid and hrefs
#pageid_to_page_links = DictStorage("pageid_to_page_links") #Storage for page links
#pageid_to_page_text = DictStorage("pageid_to_page_text") #Storage for page texts

In [169]:
def get_wiki_article_by_href(href):
    wikiart, downloaded = wiki_db.get_article_by_href(href)
    #Populate wikisyn
    for link_href, link_text in wikiart.links():
        if not link_href in wikisyn:
            wikisyn[link_href] = set()
        wikisyn[link_href].add(link_text)
        
    return wikiart

In [170]:
def save_everything():
    wiki_db.save()
    wikisyn.save()

In [171]:
def test_get_wiki_article_by_href(href):
    wikiart = get_wiki_article_by_href(href)
    save_everything()
    return wikiart

#print(test_get_wiki_article_by_href("c%2b%2b").title)


In [172]:
#Function to save page data to storages
"""def save_page_data(page_href, page_data):
    page_title = page_data['title']
    page_id = page_data['pageid']
    page_text = page_data['full'].get_text()
    
    #Register page id lookup tables
    href_to_pageid[page_href] = page_id
    href_to_pageid[page_title] = page_id
    
    #Register page title
    pageid_to_title[page_id] = page_title
    
    if not page_id in pageid_to_href:
        pageid_to_href[page_id] = set()
    pageid_to_href[page_id].add(page_href)
    pageid_to_href[page_id].add(page_title)
    
    #Register page text
    pageid_to_page_text[page_id] = page_text
    
    #Register page links and wikisyn
    page_links = get_page_links(page_data)
    pageid_to_page_links[page_id] = set()
    for link_href, link_text in page_links:
        pageid_to_page_links[page_id].add(link_href)
        if not link_href in wikisyn:
            wikisyn[link_href] = set()
        wikisyn[link_href].add(link_text)
       
    #Save everything
    href_to_pageid.save()
    pageid_to_title.save()
    pageid_to_href.save()
    pageid_to_page_text.save()
    pageid_to_page_links.save()
    wikisyn.save()"""

"def save_page_data(page_href, page_data):\n    page_title = page_data['title']\n    page_id = page_data['pageid']\n    page_text = page_data['full'].get_text()\n    \n    #Register page id lookup tables\n    href_to_pageid[page_href] = page_id\n    href_to_pageid[page_title] = page_id\n    \n    #Register page title\n    pageid_to_title[page_id] = page_title\n    \n    if not page_id in pageid_to_href:\n        pageid_to_href[page_id] = set()\n    pageid_to_href[page_id].add(page_href)\n    pageid_to_href[page_id].add(page_title)\n    \n    #Register page text\n    pageid_to_page_text[page_id] = page_text\n    \n    #Register page links and wikisyn\n    page_links = get_page_links(page_data)\n    pageid_to_page_links[page_id] = set()\n    for link_href, link_text in page_links:\n        pageid_to_page_links[page_id].add(link_href)\n        if not link_href in wikisyn:\n            wikisyn[link_href] = set()\n        wikisyn[link_href].add(link_text)\n       \n    #Save everything\n   

In [173]:
"""def get_pageid(page):
    
    #Check if the page is in the redirects table, if not, download it and register it
    if not page in href_to_pageid:
        print("Page not found. Downloading and registering it...")
        page_data = get_page_and_parse(page)
        save_page_data(page, page_data)
        print("Done.")
        
    return href_to_pageid[page]"""

'def get_pageid(page):\n    \n    #Check if the page is in the redirects table, if not, download it and register it\n    if not page in href_to_pageid:\n        print("Page not found. Downloading and registering it...")\n        page_data = get_page_and_parse(page)\n        save_page_data(page, page_data)\n        print("Done.")\n        \n    return href_to_pageid[page]'

In [174]:
#Function to download bunch of wikipedia pages at once if they are not present
check_and_download__done = 0
def check_and_download(pages):
    global check_and_download__done
    check_and_download__done = 0
    
    n_tasks = len(pages)
    
    # Function to be executed in a thread
    def download_stuff(page):
        global check_and_download__done
        try:
            get_wiki_article_by_href(page)
        except requests.exceptions.Timeout:
            print("Failed to get page " + page + ". Timed out.")
        finally:
            check_and_download__done += 1
            download_progress_hook(check_and_download__done, n_tasks)
            #percentage = round(check_and_download__done * 100 / n_tasks)
            #if percentage % 5 == 0:
                #print("{0}%.....".format(percentage), end="")
            #print("Done " + str(check_and_download__done) + "/" + str(n_tasks))

    # Instantiate a thread pool with 5 worker threads
    pool = ThreadPool(10)

    pool.map(download_stuff, pages)
    pool.wait_completion()
    
    print("\nFinishing downloading. Done tasks: " + str(check_and_download__done) + "/" + str(n_tasks))

In [175]:
#check_and_download(["Node.js"])

In [176]:
#target_page = urllib.quote("JavaScript")
#target_id = get_pageid(target_page)
#target_links = pageid_to_page_links[target_id]
#target_links_ids = set()

#for i, link in enumerate(target_links):
    #print("Working on " + link + ". " + str((i+1)) + "/" + str(len(target_links)))
    #target_links_ids.add(get_pageid(link))


In [177]:
def print_sorted_list(data, key, reverse=False):
    for k, v in sorted(data, key=key, reverse=reverse):
        print(k,v)  

In [178]:
def get_links_score(page):
    """Function to cross a list of links with a text, setting scores."""
    
    wikiart = get_wiki_article_by_href(page)
    
    pageid = wikiart.page_id
    page_links = [link_href for link_href, link_text in wikiart.links()]
    page_text = wikiart.text()
    
    links_score = dict()
    
    norm_fact = 0
    
    #Ensure all page links are present
    check_and_download(page_links)
    
    for link_href in page_links:
        links_score[link_href] = 0
        for l_text in wikisyn[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            links_score[link_href] += len(matches) 
            norm_fact += len(matches)
            
    norm_links_score = dict(map(lambda a: [a[0], float(a[1])/norm_fact], links_score.items()))
            
    return norm_links_score

#v_sum = 0
#links_score = get_links_score("MQTT")
#for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    #print(k,wikisyn[k],v)
    #v_sum += v
#print v_sum

In [179]:
#print_sorted_list(get_links_score("MQTT").items(), lambda a: a[1], True)

In [181]:
def get_node_edges_scores(page_href):
    """Function to get node edges to be placed in the graph. """
    
    edges = dict()
    
    wikiart = get_wiki_article_by_href(page_href)
    
    #Get main page data
    page_id = wikiart.page_id
    page_title = wikiart.title
    page_links = get_links_score(page_href).items()
    
    for i, link_href_score in enumerate(page_links):
        link_href = link_href_score[0]
        score = link_href_score[1]
        
        #print("Working on link {0} {1}/{2}".format(link_href, i+1, len(page_links)))
        
        link_art = get_wiki_article_by_href(link_href)
        
        link_id = link_art.page_id
        link_title = link_art.title
        
        #If there is already a title already place, sum the scores
        if (page_title, link_title) in edges:
            edges[(page_title, link_title)] += score
        else:
            edges[(page_title, link_title)] = score        
        
    return edges
    

edges_scores = get_node_edges_scores("AutoCAD")
print_sorted_list(edges_scores.items(), lambda a:a[1], True)

save_everything()

#CREATE METHOD TO CREATE GRAPH BASED ON DEEPNESS
#MAYBE PLACE STOP CONDITION TO NOT DOWNLOAD EVERY LINK
#CHECK WHETHER WIKISYN IS REALLY GOOD BECAUSE OF ERRORS. MAYBE KEEP TRACK HOW MANY TIMES EACH WORD APPEARS

....5%...10%........20%...25%........35%...40%....45%...50%...55%....60%...65%...70%....75%...80%...85%....90%...95%...100%
Finishing downloading. Done tasks: 87/87
('AutoCAD', 'Autodesk') 0.10498220640569395
('AutoCAD', 'Macintosh') 0.07829181494661921
('AutoCAD', 'MacOS') 0.051601423487544484
('AutoCAD', '.dwg') 0.0498220640569395
('AutoCAD', 'Microsoft Windows') 0.046263345195729534
('AutoCAD', 'Computer-aided design') 0.037366548042704624
('AutoCAD', 'Cloud computing') 0.02491103202846975
('AutoCAD', 'Commercial software') 0.021352313167259787
('AutoCAD', 'Android (operating system)') 0.019572953736654804
('AutoCAD', 'Portable Document Format') 0.014234875444839857
('AutoCAD', 'Italian language') 0.012455516014234875
('AutoCAD', 'IOS') 0.012455516014234875
('AutoCAD', 'Mobile app') 0.010676156583629894
('AutoCAD', 'Architecture') 0.010676156583629894
('AutoCAD', 'App Store (iOS)') 0.010676156583629894
('AutoCAD', 'Mac App Store') 0.010676156583629894
('AutoCAD', 'Dexigner') 0.01067

In [16]:
#links_score = get_page_links_score(page_links, page_data['full'].get_text())

In [15]:
#total_links = 0
#for k, v in links_score.items():
    #total_links += v

#for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    #print(k,page_links[k],float(v),float(v)/total_links)

In [138]:
#page_data = get_links_score()
#for d in page_data.iteritems():
    #print(d)

In [238]:
#print(page_data['full'].get_text())